# Project 3: Predicting the Success of a Kickstarter Campaign
A supervised learning exercise featuring logistic regression, SVM, KNN, and [Others?]

In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from flask_sqlalchemy import SQLAlchemy
import os

In [2]:
import sys
sys.executable

'/Users/brianmcmahon/anaconda3/envs/tensorflow1.4/bin/python'

In [3]:
# env variable at tensorflow1.4 per https://conda.io/docs/user-guide/tasks/manage-environments.html#saving-environment-variables
# and https://vsupalov.com/flask-sqlalchemy-postgres/

def get_env_variable(name):
    try:
        return os.environ[name]
    except KeyError:
        message = "Expected environment variable '{}' not set.".format(name)
        raise Exception(message)

# the values of those depend on your setup
POSTGRES_URL = get_env_variable("POSTGRES_URL")
POSTGRES_USER = get_env_variable("POSTGRES_USER")
POSTGRES_PW = get_env_variable("POSTGRES_PW")
POSTGRES_DB = get_env_variable("POSTGRES_DB")

In [4]:
DB_URL = 'postgresql+psycopg2://{user}:{pw}@{url}/{db}'.format(user=POSTGRES_USER,pw=POSTGRES_PW,url=POSTGRES_URL,db=POSTGRES_DB)

# app.config['SQLALCHEMY_DATABASE_URI'] = DB_URL
# app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False # silence the deprecation warning

# db = SQLAlchemy(app)

In [5]:
engine_var = DB_URL
engine = create_engine(engine_var)

In [6]:
# pd.read_sql_query('''SELECT * FROM allstarfull LIMIT 5''',engine)

In [7]:

df = pd.read_csv('data/ks-projects-201801.csv') # From kaggle dataset at https://www.kaggle.com/kemical/kickstarter-projects
df.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [8]:
df['deadline'] = pd.to_datetime(df['deadline'])
df['launched'] = pd.to_datetime(df['launched']) 
df['launched'] = df['launched'].apply(lambda x: x.strftime('%Y-%m-%d'))
df['launched'] = pd.to_datetime(df['launched']) 
df = df.sort_values(['deadline'], ascending=[False])
df['campaign_length'] = df['deadline'] - df['launched']
df['pct_goal_achieved'] = (df['usd_pledged_real'] / df['usd_goal_real'])*100
df = df.set_index('ID')
print(df.shape)
df.head()

(378661, 16)


,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,campaign_length,pct_goal_achieved
ID,,,,,,,,,,,,,,,,
22387366,Nerd Collect,Apps,Technology,GBP,2018-03-03,150000.0,2018-01-02,0.0,live,0,GB,0.0,0.0,204596.60,60 days,0.00
1036415983,Aikyam : Onnu,Music,Music,USD,2018-03-03,10000.0,2018-01-02,174.0,live,3,US,174.0,174.0,10000.00,60 days,1.74
1916988520,Back in Black Hills Movie,Drama,Film & Video,USD,2018-03-03,5500.0,2018-01-02,0.0,live,0,US,0.0,0.0,5500.00,60 days,0.00
1365286494,From the Wilderness,Drama,Film & Video,USD,2018-03-02,2500.0,2018-01-01,482.0,live,19,US,0.0,482.0,2500.00,60 days,19.28
55596200,Plateforme de statistiques,Apps,Technology,EUR,2018-03-02,9000.0,2018-01-01,0.0,live,0,FR,0.0,0.0,10923.26,60 days,0.00


In [9]:
# remove canceled, undefined, live and suspended
df = df[(df['state'] == 'successful') | (df['state'] == 'failed')]
df = pd.DataFrame(df)
start_date = pd.to_datetime('2017-06-30') 
df = df[df['launched'] >= start_date] # filter from start date to current
print(df.shape)
df.tail()   

(19291, 16)


,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,campaign_length,pct_goal_achieved
ID,,,,,,,,,,,,,,,,
315499236,Help struggling brooklyn fashion designer with...,Apparel,Fashion,USD,2017-07-07,3500.0,2017-06-30,0.0,failed,0,US,0.0,0.0,3500.0,7 days,0.000000
2003107805,Die Cut Python Stickers,Illustration,Art,USD,2017-07-07,200.0,2017-06-30,158.0,failed,30,US,0.0,158.0,200.0,7 days,79.000000
999667180,VARIANT WARS - The Prisoners,Fiction,Publishing,USD,2017-07-07,325.0,2017-06-30,425.0,successful,7,US,425.0,425.0,325.0,7 days,130.769231
1145118881,Softball Ring Box,Games,Games,USD,2017-07-07,150.0,2017-07-02,65.0,failed,1,US,65.0,65.0,150.0,5 days,43.333333
1261713735,101 Piano Songs. 1 Album,Classical Music,Music,USD,2017-07-05,1000.0,2017-07-02,33.0,failed,3,US,32.0,33.0,1000.0,3 days,3.300000


In [10]:
df.main_category = pd.Categorical(df.main_category) 
df['main_category_code'] = df.main_category.cat.codes

df.country = pd.Categorical(df.country) 
df['country_code'] = df.country.cat.codes

df.currency = pd.Categorical(df.currency) 
df['currency_code'] = df.currency.cat.codes

df.state = pd.Categorical(df.state) 
df['state_code'] = df.state.cat.codes

df.head(100)

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,campaign_length,pct_goal_achieved,main_category_code,country_code,currency_code,state_code
ID,,,,,,,,,,,,,,,,,,,,
1031685482,You Sparkle Inside: A Kids' Book + Commissions...,Children's Books,Publishing,USD,2018-01-02,2000.0,2017-12-06,6083.00,successful,133,US,1054.00,6083.00,2000.00,27 days,304.150000,12,21,13,1
1176736360,THE NEW BILLY WYLDER RECORD,Indie Rock,Music,USD,2018-01-02,10000.0,2017-11-30,11169.56,successful,177,US,650.00,11169.56,10000.00,33 days,111.695600,10,21,13,1
1894703097,Driftwood Holly ***The Venice Project***,Indie Rock,Music,EUR,2018-01-02,25000.0,2017-11-28,25417.20,successful,168,DE,0.00,30615.02,30112.50,35 days,101.668809,10,5,4,1
311863059,On a Limb's debut album,Jazz,Music,USD,2018-01-02,1000.0,2017-12-09,1743.00,successful,39,US,0.00,1743.00,1000.00,24 days,174.300000,10,21,13,1
2076121120,Briggs Barbecue Sauces,Small Batch,Food,USD,2018-01-02,200000.0,2017-11-03,1.00,failed,1,US,0.00,1.00,200000.00,60 days,0.000500,7,21,13,0
1938619867,Pizza Britt T-shirts,Crafts,Crafts,USD,2018-01-02,380.0,2017-11-13,54.00,failed,3,US,5.00,54.00,380.00,50 days,14.210526,2,21,13,0
1058353500,Nightbloom Gardens - Deathwish,Mobile Games,Games,GBP,2018-01-02,4000.0,2017-12-08,0.00,failed,0,GB,0.00,0.00,5420.64,25 days,0.000000,8,9,5,0
1117804738,THE NEW MEYTAL ALBUM,Rock,Music,USD,2018-01-02,80000.0,2017-11-17,156961.09,successful,2927,US,50905.77,156961.09,80000.00,46 days,196.201362,10,21,13,1
704593914,PAPIROARMABLES VOLUME 1 by Chakz Armada ( a pa...,Product Design,Design,MXN,2018-01-02,30000.0,2017-11-18,4250.00,failed,6,MX,181.20,219.45,1549.03,45 days,14.166930,4,15,8,0


In [ ]:
# TO DO: send to sql; retreive in "Supervised" JN
try:
    df.to_sql("kickstarter_data", engine, if_exists='replace')
except Exception as e:
    print(e)
    pass

# Temporary fix:
df.to_pickle('data/kickstarter_data.pkl')
df.to_csv('data/kickstarter_data.csv')

/Users/brianmcmahon/anaconda3/envs/tensorflow1.4/lib/python3.5/site-packages/pandas/core/generic.py:1534: UserWarning: the 'timedelta' type is not supported, and will be written as integer values (ns frequency) to the database.
  chunksize=chunksize, dtype=dtype)


In [ ]:
!scp /data/kickstarter_data.csv myaws:~ | export POSTGRES_PW

Enter passphrase for key '/Users/brianmcmahon/.ssh/id_rsa': 

In [ ]:
# try:
#     pd.read_sql_query('''DROP TABLE kickstarter_data;''',engine)
# except Exception as e:
#     print(e)
#     pass
# try:
#     pd.read_sql_query('''COPY kickstarter_data FROM 'data/kickstarter_data.csv' DELIMITER ',' CSV HEADER;''',engine)
# except Exception as e:
#     print(e)
#     pass

## State of Success

In [ ]:
df_state = df.groupby(["state"]).count().sort_values(['name'], ascending=[False])
df_state = pd.DataFrame(df_state, columns={'name'})
print(df_state.shape)
df_state

In [ ]:
df_state = df.groupby(["state"]).sum().sort_values(['usd pledged'], ascending=[False])
df_state = pd.DataFrame(df_state, columns={'usd pledged', 'usd_pledged_real','usd_goal_real'})
print(df_state.shape)
df_state

In [ ]:
df_country = df.groupby(["country"]).count().sort_values(['name'], ascending=[False])
df_country = pd.DataFrame(df_country, columns={'name'})
print(df_country.shape)
df_country

In [ ]:
df_country = df.groupby(["country"]).sum().sort_values(['usd pledged'], ascending=[False])
df_country = pd.DataFrame(df_country, columns={'usd pledged', 'usd_pledged_real','usd_goal_real'})
print(df_country.shape)
df_country

In [ ]:
df_currency = df.groupby(["currency"]).count().sort_values(['name'], ascending=[False])
df_currency = pd.DataFrame(df_currency, columns={'name'})
print(df_currency.shape)
df_currency

In [ ]:
df_currency = df.groupby(["currency"]).sum().sort_values(['usd pledged'], ascending=[False])
df_currency = pd.DataFrame(df_currency, columns={'usd pledged', 'usd_pledged_real','usd_goal_real'})
print(df_currency.shape)
df_currency

In [ ]:
df_main_category = df.groupby(["main_category"]).count().sort_values(['name'], ascending=[False])
df_main_category = pd.DataFrame(df_main_category, columns={'name'})
print(df_main_category.shape)
df_main_category

In [ ]:
df_main_category = df.groupby(["main_category"]).sum().sort_values(['usd pledged'], ascending=[False])
df_main_category = pd.DataFrame(df_main_category, columns={'usd pledged', 'usd_pledged_real','usd_goal_real'})
print(df_main_category.shape)
df_main_category

In [ ]:
df_category = df.groupby(["category"]).count().sort_values(['name'], ascending=[False])
print(df_category.shape)
df_category

In [ ]:
df_category = df.groupby(["category"]).sum().sort_values(['usd pledged'], ascending=[False])
df_category = pd.DataFrame(df_category, columns={'usd pledged', 'usd_pledged_real','usd_goal_real'})
print(df_category.shape)
df_category